In [ ]:
!pip install cassio
!pip install tiktoken
!pip install datasets
!pip install langchain
!pip install openai
!pip install -U langchain-community
!pip install PyPDF2

Imports:

In [17]:
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from datasets import load_dataset
from PyPDF2 import PdfReader
import cassio
from typing_extensions import Concatenate
import os

define keys

In [6]:
os.environ['OPENAI_API_KEY'] = '<enter api key>'
os.environ['ASTRA_DB_ID'] = '62db5132-7ca6-4bd4-8f29-a7f7bf3abc33'
os.environ['ASTRA_DB_APP_TOKEN'] = 'AstraCS:nusoefSiysYweSRTbWQyCRhM:b3c5a2745a8194aaae6dfb63126aa9f9d4d13ee0d9a5799e0e0b93d0f1626ff7'

In [4]:
pdf = PdfReader('testspeech.pdf')

In [ ]:
raw = ''
for x, page in enumerate(pdf.pages):
  text = page.extract_text()
  if text:
    raw += text
print(raw)

conenct to db

In [ ]:
cassio.init(token=os.environ['ASTRA_DB_APP_TOKEN'], database_id=os.environ['ASTRA_DB_ID'])

In [ ]:
model = OpenAI(openai_api_key=os.environ['OPENAI_API_KEY'])
embedding_model = OpenAIEmbeddings(openai_api_key=os.environ['OPENAI_API_KEY'])

In [16]:
vectorstore = Cassandra(
    embedding=embedding_model,
    table_name = "pdf_qa",
    session=None,
    keyspace=None
)

In [18]:
splitter = CharacterTextSplitter(
    separator = '\n',
    chunk_size = 800,
    chunk_overlap = 200,
    length_function = len
)

chunks = splitter.split_text(raw)

load data in vector db

In [21]:
vectorstore.add_texts(chunks[:100])
vectorstoreindex = VectorStoreIndexWrapper(vectorstore=vectorstore)

inserted 10 headlines.


q and a

In [ ]:
q1 = True
while True:
  if q1:
    query = input("\nAsk a question (or type 'q' to exit the prompt)!").strip()
  else:
    query = input("\nAsk another question (or type 'q' to exit the prompt)!").strip()

  if query.lower() == 'q':
    break
  if query == '':
    continue
  q1 = False
  print("\nquestion is : \"%s\"" % query)
  a = vectorstoreindex.query(query, llm=model)
  print("\nanswer is: \"%s\"\n" % a)
  print("here are the relevant documents, sorted by relevance to your question:")
  for doc, score in vectorstore.similarity_search_with_score(query, k=4):
    print("    [%0.4f] \"%s ...\"" % (score, doc.page_content[:84]))